In [1]:
# Thsi is the code used for the paper results
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import os
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct
import matplotlib.pyplot as plt


In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 10,
    "exp_name": ct.generate_readable_short_id("calibsize_final"),
    "path": "../../",
    "split": "random_split", #CV, random_split
    "cv_folds": 10,

    "test_split": 0.0588,
    "calib_split": 0.5,
    
    "plot": False,
    "calib_methods": ["RF_d", "RF_opt", "RF_large",
                      "Platt", "ISO", "Beta", "PPA", "VA",
                      "Rank"#, "Elkan", "tlr",                      
                      ],
    
    "calib_method_colors": {
        "RF_d": "blue", 
        "RF_opt": "orange", 
        "RF_large": "red",
        "Platt": "Brown", 
        "ISO": "purple", 
        "Beta": "magenta", 
        "PPA": "olive", 
        "VA": "gray",
        "Rank": "silver"     
    },
    
    "metrics": ["acc", "brier", "ece", "logloss", "tce"], 

    #data
    "data_name": "synthetic",
    "plot_data": True,
    "data_size": 1700,
    "n_features": 2,

    "bais_accuracy": 0,
    
    "class1_mean_min":0, 
    "class1_mean_max":1,
    "class2_mean_min":1, 
    "class2_mean_max":3, 

    "class1_cov_min":1, 
    "class1_cov_max":2,
    "class2_cov_min":1, 
    "class2_cov_max":2, 


    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth": np.arange(2, 100).tolist(),
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf": np.arange(1, 11).tolist(),
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False],
                    },
    
    "oob": False,
    "laplace": 0, # if set to 1, OOB must be false

    "exp_key": "calib_size",
    "exp_values":  np.arange(2, 100, 2).tolist()
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param 2 done
exp_param 4 done
exp_param 6 done
exp_param 8 done
exp_param 10 done
exp_param 12 done
exp_param 14 done
exp_param 16 done
exp_param 18 done
exp_param 20 done
exp_param 22 done
exp_param 24 done
exp_param 26 done
exp_param 28 done
exp_param 30 done
exp_param 32 done
exp_param 34 done
exp_param 36 done
exp_param 38 done
exp_param 40 done
exp_param 42 done
exp_param 44 done
exp_param 46 done
exp_param 48 done
exp_param 50 done
exp_param 52 done
exp_param 54 done
exp_param 56 done
exp_param 58 done
exp_param 60 done
exp_param 62 done
exp_param 64 done
exp_param 66 done
exp_param 68 done
exp_param 70 done
exp_param 72 done
exp_param 74 done
exp_param 76 done
exp_param 78 done
exp_param 80 done
exp_param 82 done
exp_param 84 done
exp_param 86 done
exp_param 88 done
exp_param 90 done
exp_param 92 done
exp_param 94 done
exp_param 96 done
exp_param 98 done


In [4]:
plot_calib_methods = params["calib_methods"].copy()
# plot_calib_methods.remove("RF_large")
# plot_calib_methods.remove("Elkan")
# plot_calib_methods.remove("RF")
plot_calib_methods

['RF_d', 'RF_opt', 'RF_large', 'Platt', 'ISO', 'Beta', 'PPA', 'VA', 'Rank']

In [11]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=False, 
                                    std=True)

path = f"results/{params['exp_name']}/calib_size"
if not os.path.exists(path):
    os.makedirs(path)

max_value = [0.81, 0.2, 0.15, 4, 0.1]
min_value = [0.72, 0.14, 0.025, 0, 0.015]
for metric, max_v, min_v in zip(params["metrics"], max_value, min_value): #
    ax = tables[metric][plot_calib_methods].plot(color=params["calib_method_colors"])
    ax.set_xlabel(params["exp_key"])
    ax.set_ylabel(metric)
    plt.ylim(min_v, max_v)
    plt.savefig(f"{path}/cs_{metric}.pdf", format='pdf', transparent=True)
    plt.close() 
